In [9]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import requests
import textwrap

In [4]:
dataset = load_dataset("squad")
print(dataset)
print("\nColumns:", dataset["train"].column_names)
print("\nExample from train split:\n")
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Columns: ['id', 'title', 'context', 'question', 'answers']

Example from train split:

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end o

In [5]:
def curate_dataset(split):
    curated = []
    for ex in dataset[split]:
        curated.append({
            "context": ex["context"].strip(),
            "question": ex["question"].strip(),
            "answers": ex["answers"]
        })
    return curated

train_data = curate_dataset("train")
val_data = curate_dataset("validation")

print(f"Train examples: {len(train_data)}, Validation examples: {len(val_data)}")
print("Example:")
print(val_data[0])

Train examples: 87599, Validation examples: 10570
Example:
{'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.', 'question': 'Which NFL team represented the AFC at Super Bowl 50?', 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answ

In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

C:\Users\Alexandru\Desktop\DL4NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alexandru\.cache\huggingface\hub\models--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,)

In [11]:
def reconstruct_text(tokens):
    output = []
    for tok in tokens:
        if tok.startswith("##") and output:
            output[-1] += tok[2:]
        else:
            output.append(tok)
    return " ".join(output)

In [12]:
@torch.no_grad()
def get_answer(question, context):
    input_ids = tokenizer.encode(question, context)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0] * num_seg_a + [1] * num_seg_b

    start_scores, end_scores = model(
        torch.tensor([input_ids]).to(device),
        token_type_ids=torch.tensor([segment_ids]).to(device),
        return_dict=False
    )

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer_tokens = tokens[start_index:end_index + 1]
    answer = reconstruct_text(answer_tokens)

    return answer.strip()

In [13]:
for sample in val_data[:3]:
    question = sample["question"]
    context = sample["context"]
    predicted = get_answer(question, context)

    print("\n" + "-"*100)
    print("Question:", question)
    print("Predicted answer:", predicted)
    print("True answers:", sample["answers"]["text"])


----------------------------------------------------------------------------------------------------
Question: Which NFL team represented the AFC at Super Bowl 50?
Predicted answer: denver broncos
True answers: ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']

----------------------------------------------------------------------------------------------------
Question: Which NFL team represented the NFC at Super Bowl 50?
Predicted answer: carolina panthers
True answers: ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers']

----------------------------------------------------------------------------------------------------
Question: Where did Super Bowl 50 take place?
Predicted answer: levi ' s stadium in the san francisco bay area at santa clara , california
True answers: ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]


In [14]:
def translate_libre(text):
    try:
        r = requests.post(
            "https://libretranslate.com/translate",
            data={"q": text, "source": "en", "target": "ro", "format": "text"},
            timeout=10
        )
        if r.ok:
            return r.json().get("translatedText")
    except Exception:
        pass
    return None

def translate_mymemory(text):
    try:
        r = requests.get(
            "https://api.mymemory.translated.net/get",
            params={"q": text, "langpair": "en|ro"},
            timeout=10
        )
        if r.ok:
            return r.json().get("responseData", {}).get("translatedText")
    except Exception:
        pass
    return None

def translate_to_ro(text):
    if not text.strip():
        return text
    t = translate_libre(text)
    if t:
        return t
    t = translate_mymemory(text)
    if t:
        return t
    return text


In [16]:
sample = val_data[7]
question = sample["question"]
context = sample["context"]

answer_en = get_answer(question, context)
answer_ro = translate_to_ro(answer_en)

print("Question:", question)
print("Answer (EN):", answer_en)
print("Answer (RO):", answer_ro)

Question: What is the AFC short for?
Answer (EN): american football conference
Answer (RO): conferință de fotbal american
